## Install the package dependencies before running this notebook

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob

"""
    number of trajectories in each city
    # austin --  train: 43041 test: 6325 
    # miami -- train: 55029 test:7971
    # pittsburgh -- train: 43544 test: 6361
    # dearborn -- train: 24465 test: 3671
    # washington-dc -- train: 25744 test: 3829
    # palo-alto -- train:  11993 test:1686

    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds
    
"""

'\n    number of trajectories in each city\n    # austin --  train: 43041 test: 6325 \n    # miami -- train: 55029 test:7971\n    # pittsburgh -- train: 43544 test: 6361\n    # dearborn -- train: 24465 test: 3671\n    # washington-dc -- train: 25744 test: 3829\n    # palo-alto -- train:  11993 test:1686\n\n    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds\n    \n'

## Create a Torch.Dataset class for the training dataset

In [51]:
from glob import glob
import pickle
import numpy as np

ROOT_PATH = "./"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

def get_city_trajectories(city="palo-alto", split="train", normalized=False):
    f_in = ROOT_PATH + split + "/" + city + "_inputs"
    inputs = pickle.load(open(f_in, "rb"))
    inputs = np.asarray(inputs)
    
    outputs = None
    mean, std = None, None

    if split=="train":
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)

        trajectories = np.concatenate((inputs, outputs), axis=1)
        trajectories = trajectories.reshape(-1, trajectories.shape[1], trajectories.shape[2])
        trajectories = trajectories.astype(np.float32)

        if normalized:
            mean = np.mean(trajectories, axis=0)
            std = np.std(trajectories, axis=0)
            trajectories = (trajectories - mean) / std

        inputs = []
        outputs = []
        for trajectory in trajectories:
            for i in range(trajectory.shape[0] - 50):
                inputs.append(trajectory[i:i+50])
                outputs.append(trajectory[i+50])

    inputs, outputs = np.asarray(inputs), np.asarray(outputs)
    return inputs, outputs, mean, std

class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, city: str, split:str):
        super(ArgoverseDataset, self).__init__()

        self.inputs, self.outputs, self.mean, self.std = get_city_trajectories(city=city, split=split, normalized=True)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):

        data = (self.inputs[idx], self.outputs[idx])

        return data
    
    def get_mean_std(self):
        return self.mean, self.std

# intialize each dataset
train_austin = ArgoverseDataset(city = "austin", split = "train")
train_miami = ArgoverseDataset(city = "miami", split = "train")
train_palo_alto = ArgoverseDataset(city = "palo-alto", split = "train")
train_pittsburgh = ArgoverseDataset(city = "pittsburgh", split = "train")
train_dearborn = ArgoverseDataset(city = "dearborn", split = "train")
train_washington_dc = ArgoverseDataset(city = "washington-dc", split = "train")

austin_mean, austin_std = train_austin.get_mean_std()[0][0]
miami_mean, miami_std = train_miami.get_mean_std()[0][0]
palo_alto_mean, palo_alto_std = train_palo_alto.get_mean_std()[0][0]
pittsburgh_mean, pittsburgh_std = train_pittsburgh.get_mean_std()[0][0]
dearborn_mean, dearborn_std = train_dearborn.get_mean_std()[0][0]
washington_dc_mean, washington_dc_std = train_washington_dc.get_mean_std()[0][0]

test_austin = ArgoverseDataset(city = "austin", split = "test")
test_miami = ArgoverseDataset(city = "miami", split = "test")
test_palo_alto = ArgoverseDataset(city = "palo-alto", split = "test")
test_pittsburgh = ArgoverseDataset(city = "pittsburgh", split = "test")
test_dearborn = ArgoverseDataset(city = "dearborn", split = "test")
test_washington_dc = ArgoverseDataset(city = "washington-dc", split = "test")

# Train Validation Split
train_austin, val_austin = torch.utils.data.random_split(train_austin, [len(train_austin) - len(train_austin)//5, len(train_austin)//5])
train_miami, val_miami = torch.utils.data.random_split(train_miami, [len(train_miami) - len(train_miami)//5, len(train_miami)//5])
train_palo_alto, val_palo_alto = torch.utils.data.random_split(train_palo_alto, [len(train_palo_alto) - len(train_palo_alto)//5, len(train_palo_alto)//5])
train_pittsburgh, val_pittsburgh = torch.utils.data.random_split(train_pittsburgh, [len(train_pittsburgh) - len(train_pittsburgh)//5, len(train_pittsburgh)//5])
train_dearborn, val_dearborn = torch.utils.data.random_split(train_dearborn, [len(train_dearborn) - len(train_dearborn)//5, len(train_dearborn)//5])
train_washington_dc, val_washington_dc = torch.utils.data.random_split(train_washington_dc, [len(train_washington_dc) - len(train_washington_dc)//5, len(train_washington_dc)//5])



## Create a DataLoader class for training

In [52]:
batch_sz = 10  # batch size 
train_loader = DataLoader(train_palo_alto,batch_size=batch_sz,shuffle=True)
val_loader = DataLoader(val_palo_alto,batch_size=batch_sz,shuffle=True)
test_loader = DataLoader(test_palo_alto,batch_size=batch_sz,shuffle=True)


## Sample a batch of data and visualize 

In [53]:
import matplotlib.pyplot as plt
import random


def show_sample_batch(sample_batch):
    """visualize the trajectory for a batch of samples"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)

    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i,:,0], inp[i,:,1])
        axs[i].scatter(out[i,0], out[i,1], c='r')
    

for i_batch, sample_batch in enumerate(train_loader):
    if i_batch != 50: continue
    inputs, outputs = sample_batch
    show_sample_batch(sample_batch)
    break




## Create Model

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

from torch import nn

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=2, hidden_size=512, num_layers=3, batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(512, 256)
        self.tanh = nn.Tanh()
        self.linear2 = nn.Linear(256, 2)
        self.leakyrelu = nn.LeakyReLU()

    def forward(self, x, h=None, c=None):
        if (h is not None) and (c is not None):
            x, (h, c) = self.lstm(x, (h, c))
        else:
            x, (h, c) = self.lstm(x)
        x = self.dropout(x)
        x = self.linear(x)
        x = self.tanh(x)
        x = self.linear2(x)
        return x, h, c

    def predict(self, x, h=None, c=None):
        output, h, c = self.forward(x, h, c)
        output = output[-1]
        output = output.cpu().detach().numpy()
            
        return output, h, c
        

Using device: cpu


## Train Model

In [54]:
num_epochs = 5
learning_rate = 0.001

lstm = LSTM()
lstm.to(device)
optimizer = torch.optim.Adam(lstm.parameters(), learning_rate)
loss_fn = nn.MSELoss()

for epoch in range(num_epochs):
    lstm.train(True)
    for i_batch, sample_batch in enumerate(train_loader):
        inp, label = sample_batch
        
        inp = inp.to(device)
        output, h, c = lstm(inp.float())

        output = output[:, -1, :]

        loss = loss_fn(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i_batch % 10000 == 0 and i_batch != 0:
            print(f"Epoch: {epoch}/{num_epochs}, Batch: {i_batch}/{len(train_loader)}, Loss: {loss.item()}")

    lstm.train(False)
    with torch.no_grad():
        for i_batch, sample_batch in enumerate(val_loader):
            inp, label = sample_batch
            inp = inp.to(device)
            output, h, c = lstm(inp.float())
            output = output[:, -1, :]
            loss = loss_fn(output, label)
            print(f"Epoch: {epoch}/{num_epochs}, Batch: {i_batch}/{len(val_loader)}, Loss: {loss.item()}")
            break

torch.save(lstm.state_dict(), "lstm_model.pt")

        

Epoch: 0/5, Batch: 0/57567, Loss: 0.5454278588294983
Epoch: 0/5, Batch: 10/57567, Loss: 0.13322672247886658


KeyboardInterrupt: 